In [ ]:
import os, re, matplotlib, pandas, collections, importlib
import matplotlib.pyplot as plt
import numpy as np
import scipy
import scipy.stats
import pandas, re
import seaborn as sns
import scipy as sp
from typing import List, Union, Mapping, Tuple
from pprint import pprint
from scipy import stats

import heatmaper as hm
import xlLoader
import stabilityLoader
importlib.reload(stabilityLoader)
pma_dir = '/Users/dp/pma/'

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rc('font',**{'family':'sans-serif','sans-serif':['Arial']})

In [ ]:
from statistics import NormalDist
from scipy.stats import norm

def confidence_interval(data, confidence=0.95):
    #https://stackoverflow.com/questions/15033511/compute-a-confidence-interval-from-sample-data
    if len(data) < 2:
        return np.nan
    dist = NormalDist.from_samples(data)
    z = NormalDist().inv_cdf((1 + confidence) / 2.)
    h = dist.stdev * z / ((len(data) - 1) ** .5)
    return dist.mean - h, dist.mean + h

In [ ]:
importlib.reload(xlLoader)




        
def get_xl_pval_df_from_ttest_vs_wt(xl):
    mut_to_pval = {}
    mut_to_ci = {}

    for prot in set([_ for _ in xl.Protein if ' ' in _]):
        sub = xl.loc[[x==prot for x in xl.Protein], 'Value vs WT'].to_numpy()
        res = scipy.stats.ttest_1samp(sub, 0.)
        #ci = confidence_interval(sub)
        mut_to_pval[prot] = np.log10(res.pvalue)
        #mut_to_ci[prot] = ci

    for prot in [_ for _ in xl.Protein if ' ' not in _]:
        mut_to_pval[prot] = 1.

    xl_pval_df = pandas.DataFrame.from_dict(mut_to_pval, 'index').T
    xl_pval_df.index = ['P value']
    
    #ci_df = pandas.DataFrame.from_dict(mut_to_ci, 'index').T
    return xl_pval_df


def sorted_gids(gid, xl):
    _df = xl.loc[[bool(x==gid[1]) for x in xl.Protein], :]
    gids = list(set(_df.loc[:, 'Group_ID']))
    to_exp_decimal = {(exp, protein, rep): float(str((exp.split('xp')[-1])) + '.' + str(rep.split(' ')[-1])) \
                      for (exp, protein, rep) in gids}
    gids = sorted(gids, key=lambda x: to_exp_decimal[x])
    return gids

def sorted_all_gids_with_the_same_wt_protein(protein, xl):
    _df = xl.loc[[bool(  x.split(' ')[0]==protein.split(' ')[0]  ) for x in xl.Protein], :]
    gids = list(set(_df.loc[:, 'Group_ID']))
    to_exp_decimal = {(exp, protein, rep): float(exp.split('xp')[-1]) for (exp, protein, rep) in gids}
    gids = sorted(gids, key=lambda x: to_exp_decimal[x])
    return gids


def define_all_batches(xl):
    to_batches = {}
    for protein in set(xl['Protein']):
        to_batches[protein] = batches_for_a_protein(protein, xl)
    xl['Batch'] = [to_batches[protein][exp, rep] for protein, exp, rep in zip(
        xl.Protein, xl.Exp, xl.Rep)]
    
def batches_for_a_protein(protein, xl):

    _sorted = sorted_all_gids_with_the_same_wt_protein(protein, xl)
    exp_rep_pairs = [(_exp, _rep) for _exp, _prot, _rep in _sorted]
    # Remove duplicates keeping list order.
    _ = [] 
    [_.append(x) for x in exp_rep_pairs if x not in _] 
    exp_rep_pairs = _
    to_batch = {exp_rep_pair:n for n, exp_rep_pair in enumerate(exp_rep_pairs, start=1)}
    return to_batch

def batch1(gid, xl):
    return xl.loc[[bool(prot==gid[1] and batch==1) for prot, batch in zip(xl.Protein, xl.Batch)], :]
    
def subtract_pair_from_batch1(gid, xl):
    val = xl.loc[[groupid==gid for groupid in xl.Group_ID], 'vs_protein_mean'].to_numpy()[0]    
    val_batch1 = batch1(gid, xl)
    if len(val_batch1.index):
        val_batch1 = val_batch1['vs_protein_mean'].to_numpy()[0]
        return val - val_batch1
    print(f"No batch1 found for {gid}")
    return np.nan

def pair_batches(xl):
    b2 = xl.loc[[x!=1 for x in xl.Batch], :]
    arr = []
    for protein in [x for x in set(b2['Protein']) if ' ' in x]:  # For each mutant.
        sub = b2.loc[[x==protein for x in b2.Protein], '-batch1'].to_numpy()
        wt = b2.loc[[x==protein.split(' ')[0] for x in b2.Protein], '-batch1'].to_numpy()
        for n, val in enumerate(sub):
            if len(wt) > n and not (np.isnan(wt[n]) or np.isnan(sub[n])):
                arr.append([wt[n], sub[n]])
    return arr


def get_xl_pval_df_from_norm(xl, mean, stddev):
    mut_to_pval = {}
    
    n = norm(loc=mean, scale=stddev)#.cdf([1.3, 1.3])
    #print(np.product([1-v for v in a]))
    for prot in set([_ for _ in xl.Protein if ' ' in _]):
        sub = xl.loc[[x==prot for x in xl.Protein], 'vs_protein_mean'].to_numpy()
        
        #for val in sub:
        #    print(val, norm(loc=mean, scale=stddev).cdf(3.))
            
        print(sub)
        
        # pvals for increase.
        pval_inc = 1-n.cdf(np.mean(sub))

        # pvals for decrease.
        pval_dec = n.cdf(np.mean(sub))
        
        lower = min([pval_inc, pval_dec])
        
        print(f"{prot}: {pval_inc}, {pval_dec}")
        
#get_xl_pval_df_from_norm(xl, mean, stddev)
xl_pval_df = get_xl_pval_df_from_ttest_vs_wt(xl)
#print(xl_pval_df)

In [ ]:
# Load cross-linking data.
xlLoad = xlLoader.xlLoader(f"{pma_dir}/percentCrosslinked.xlsx")
(df, recurrent) = xlLoad.load()
xl = recurrent.loc[recurrent['Label']=='% XL (minimal region)',:].copy()
xlLoad.as_fraction_of_wt(xl)

# To investigate batch effects, define batches.
define_all_batches(xl)

# Normalize all the the WT protein mean.
# xl['vs_protein_mean'] = XL/(average WT protein XL)
prot_means = xl.groupby(by=['Protein'])['Value'].mean().to_dict()
xl['vs_protein_mean'] = [(val)/max([0.001, prot_means[p.split(' ')[0]]]) for val, p, exp in zip(xl['Value'], xl['Protein'], xl.Exp)]

# Use batch definitions to get the change in signal from batch 1 for each protein.
# xl['-batch1'] = (XL{batch x} - XL{batch 1})/(average WT protein XL)
xl['-batch1'] = [subtract_pair_from_batch1(gid, xl) for gid in xl.Group_ID]

# Pair (WT, MUT) for each batch for each protein (of -batch1 values, batch1 itself not included).
# These are [(XL{WT batch 2} - XL{WT batch 1})/(average WT protein XL),
#           (XL{MUT batch 2} - XL{MUT batch 1})/(average WT protein XL)].
wt_mut_pairs = pair_batches(xl)

# Create df for sns.lmplot.
arrdf = pandas.DataFrame(wt_mut_pairs)
arrdf.columns = ['WT', 'MUT']

# Get the parameters for a linear regression.
x=[x[0] for x in wt_mut_pairs]
y=[x[1] for x in wt_mut_pairs]
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x, y)
print(f"slope={slope}, intercept={intercept}, R={r_value}, p={p_value}, std_err={std_err}, R^2={r_value**2}")
#plt.scatter(x=[x[0] for x in arr], y=[x[1] for x in arr], c='k', alpha=0.3)

# sns.lmplot the -batch1 values.
sns.lmplot(x='WT', y='MUT', data=arrdf)
plt.show()
plt.clf()
plt.close()

############
# Get a Protein->average value dict.
_ = xl.groupby(by=['Protein'])['Value'].mean().to_dict()

# Subset to proteins with a sufficiently high WT protein mean value.
xl = xl.loc[[prot_means[p]>0.1 for p in xl['WT protein']], :]

# log2 transform
xl['vs_protein_mean'] = np.log2(xl['vs_protein_mean'])
xl = xl.loc[[np.isfinite(x) for x in xl['vs_protein_mean']], :]

# Plot val/(WT mean), including outliers.
ax = sns.distplot(xl['vs_protein_mean'], kde=True)

# Subset to val/(WT mean) below 2 to remove outliers, get the mean and std dev, and fit a normal distribution.
vm = xl['vs_protein_mean'].to_numpy()
vm = vm[vm<2]
vm = vm[vm>-2]
mean, stddev = (np.mean(vm), np.var(vm)**0.5)
fit_norm_vals = np.random.normal([np.mean(vm)], [np.var(vm)**0.5], size=1000).T

# Print mean/std dev.
print(f"mean={mean}, stddev={stddev}")

# Plot the fit normal distribution.
sns.distplot(fit_norm_vals, ax=ax, hist=False, fit=norm, kde=False)
plt.show()
plt.clf()
plt.close()

# Define batch_ID column.
q = xl.groupby(by=['Protein'])['Group_ID']
print(q.get_group('A1CF').to_list())
xl['batch_ID'] = [q.get_group(x).to_list() + q.get_group(x.split(' ')[0]).to_list() for x in xl.Protein]
print(xl.head())

In [ ]:
xlLoad = xlLoader.xlLoader(f"{pma_dir}/percentCrosslinked.xlsx")
(df, recurrent) = xlLoad.load()
sns.

In [ ]:
# Determine p values from t statistics and add to xl dataframe.


# Load cross-linking data.
xlLoad = xlLoader.xlLoader(f"{pma_dir}/percentCrosslinked.xlsx")
(df, recurrent) = xlLoad.load()
xl = recurrent.loc[recurrent['Label']=='% XL (minimal region)',:].copy()
xlLoad.as_fraction_of_wt(xl)

# To investigate batch effects, define batches.
define_all_batches(xl)
def add_pvalues_using_var_across_all_samples(xl):
    # Normalize all to the WT protein mean.
    # xl['vs_protein_mean'] = XL/(average WT protein XL)
    # Not used here currently.
    prot_means = xl.groupby(by=['Protein'])['Value'].mean().to_dict()
    xl['vs_protein_mean'] = [(val)/max([0.001, prot_means[p.split(' ')[0]]]) for val, p, exp in zip(xl['Value'], xl['Protein'], xl.Exp)]

    # Do we need this?
    xl = xl.loc[[np.isfinite(x) for x in xl['log2 Value vs WT']], :]

    # Only sufficiently high values used for determining variance, and remove WTs (all WT = 1).
    mut_only = xl.loc[[x>0.1 for x in xl['Value']], :]
    mut_only = mut_only.loc[[' ' in x for x in mut_only.Protein], :]

    value_vs_wt_by_prot_mut_only = mut_only.groupby('Protein')['Value vs WT'].apply(np.array)
    deltas = []
    for x in value_vs_wt_by_prot_mut_only:
        [deltas.append(x[n] - x[0]) for n in range(1, len(x))]

    print(f"Estimating total variance from {len(deltas)} values of MUT/WT. These are deltas of MUT/max([WT, 10**-6])).")
    print(f"Each value is MUT(batch x)/WT(batch x) - MUT(batch 1)/WT(batch 1).")
    print(f"These deltas are: {deltas}.")

    variance = np.var(deltas)
    print(f"variance={variance}, mean={np.mean(deltas)}")

    def t_statistic(arr, variance):
        n = len(arr)
        t = np.mean(arr) - 1
        sample_var = (variance + np.var(arr))/2  # Will this be valid?
        t = t/(sample_var/np.sqrt(n))
        pval = stats.t.sf(np.abs(t), n-1)*2
        #print(f"t-statistic = {t:6.3f} pvalue = {pval:6.4f}")
        return t, pval

    value_vs_wt_by_prot = xl.groupby('Protein')['Value vs WT'].apply(np.array)
    mut_to_pval = {}
    for name, val in zip(value_vs_wt_by_prot.index, value_vs_wt_by_prot):
        mut_to_pval[name] = t_statistic(val, variance)[1]
        print(f"{name}, {val}, pvalue = {mut_to_pval[name]:6.4f}")
        mean_wt_xl = prot_means[name.split(' ')[0]]
        if mean_wt_xl < 0.01:
            mut_to_pval[name] = 1.
            print(f"--> set pval to 1. because XL rate was < 0.01% (unreliable).")

    # Setting this column is the goal of this function:
    xl['P value'] = [mut_to_pval.get(name, 1.) for name in xl.Protein]

    # Diagnostic plot:
    ax = sns.distplot(
        deltas,
        #fit=norm,#
        #xl['log2 Value vs WT'], kde=True
    )
    delta_norm = np.random.normal([np.mean(deltas)], [np.var(deltas)**0.5], size=1000).T

    sns.distplot(delta_norm, fit=norm, hist=False, kde=False, ax=ax)
    plt.show()
    plt.clf()
    plt.close()
    
add_pvalues_using_var_across_all_samples(xl)


### 